<h2>Fusion corrective RAG with parent/child strategy</h2>

In [1]:
#No ejecutar
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from langchain_community.vectorstores import Chroma
from langchain.vectorstores import VectorStore 
from langchain_core.documents.base import Document
from langchain.load import dumps, loads


from langchain_core.retrievers import BaseRetriever
from langchain.callbacks.manager import CallbackManager
from typing import List
from langchain_core.callbacks import CallbackManagerForRetrieverRun

from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain_core.output_parsers import JsonOutputParser
#import spacy

In [6]:
import requests

# Credenciales de autenticación

usuario = "joseacentic"
passw = "20centic05"

# Prueba la conexión
response = requests.get("http://80.73.154.76:18001/ollama", auth=(usuario, passw))

print("Status Code:", response.status_code)
print("Response Text:", response.text)


ConnectTimeout: HTTPConnectionPool(host='80.73.154.76', port=18001): Max retries exceeded with url: /ollama (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x700ffee4a710>, 'Connection to 80.73.154.76 timed out. (connect timeout=None)'))

In [8]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = Ollama(base_url="http://10.7.15.205:11434/",
             model="qwen2.5:14b",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

#llm.invoke('Hola')

template = """ 
## Rol 
You are an apparel shopping assistant.

You **must always** follow these rules:

Use Markdown and emojis to enhance readability.
Never mention or discuss other e-commerce platforms.
Always use currency {tenant_currency} with two decimal places when referencing prices.
Your output must always be formatted in JSON using the keys "response", "isAnswered", and "isLicitQuestion".

## Context Analysis:
You will evaluate the context provided below to determine whether the buyer's question is licit:

**Product Details**:

Name: {product_name}
Description: {product_summaryDescription}
Price: {product_unitPrice}
Buyer Question: {question}

**Rules for Licit Questions:**
A question is licit if it pertains to the product's characteristics, features, pricing, availability, or other directly relevant details.
If the question is **not licit**, your response **must only be**: "I can't help you with that".
**JSON Output:**
"response": A concise, friendly answer to the buyer's question, formatted with Markdown. This field is mandatory only if "isLicitQuestion": true. 
"isAnswered": true if the question is licit, otherwise false.
"isLicitQuestion": true if the question is licit, otherwise false.

## Step-by-step Process:

1. Evaluate if the question is licit:
      If yes, generate a detailed response using product details.
      
      If no, output only:
{
    "response": "I can't help you with that",
    "isAnswered": false,
    "isLicitQuestion": false
}
2.Format your output strictly in JSON as described above.
"""

prompt = PromptTemplate(
    input_variables=["tenant_currency", "product_name","product_summaryDescription","product_unitPrice", "question","tenant_locale"],
    template=template,
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
)

pname = "Wool Knit Scarf"
pdescrp = "Cozy wool knit scarf to keep you warm during the chilly months."
pprize = "29.99"
question = "What is the weather like in Paris?"
llm_chain.run(input={"tenant_currency":"", "product_name":pname,"product_summaryDescription":pdescrp,"product_unitPrice":pprize, "question":question,"tenant_locale":""})


/tmp/ipykernel_96187/1480801614.py:8: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(base_url="http://10.7.15.205:11434/",


ValueError: Missing some input keys: {'question', 'product_unitPrice', 'tenant_currency', 'product_summaryDescription', '\n    "response"', 'product_name'}

In [2]:
from langchain_community.document_loaders import PyPDFLoader
myFile = 'annualreport.pdf'
loader = PyPDFLoader("DATA/" + myFile)

data = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    length_function = len
)

child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        length_function=len
    )

<u>Nota:</u> 

Ver si es posible unir las dos celdas siguientes en una única base de datos añadiendo una nueva colección de documentos. 

No está claro por la forma en la que se tiene que acceder a los documentos de la base de datos desde el retriever personalizado. 

<del>IMPORTANTE: Comprobar que cuando se define el store = InMemoryStore() dos veces (una para cada modelo de embeddings) tiene la misma referencia (doc_id en el metadata) porque si no estaremos recuperando documentos iguales mas de una vez. </del> Tienen referencias distintas

<del>OJO puede ser que no haya ningun problema puesto que dentro de la Store se guarde el mismo documento dos veces con dos ids diferentes. (Probar) </del> Esto es lo que ocurre

<h4><b>Problema en el reranking al meter el parent child: <u> SOLUCIONADO  </u> </b></h4>  

Cuando no se usaba el parent child no habia problema al usar el dumnps(doc) para comprobar los que eran iguales y los que no. Ahora como se generan doc_id diferentes en función de las vectorstore si dos documentos iguales vienen de diferentes bases de datos
el dumps() no los asocia como iguales puesto que no tienen el mismo metadata. entonces en el reranking se meten de nuevo a 0 y no suma puntuación si ya está metido en la lista 


Se arreglaría todo metiendo en la vectorstore nueva los documentos con los mismos doc_id que en la bd anterior. --- > No se ha solucionado así

In [4]:
from langchain_community.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

vectorstore = Chroma(
        collection_name="documents",
        persist_directory="DATA/chroma_db",
        embedding_function=hugg_embeddings
    )

store = InMemoryStore()

full_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

full_retriever.add_documents(data, ids = None)

/tmp/ipykernel_121947/954869930.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [5]:
vectorstore_bgem3 = Chroma(
        collection_name="documents",
        persist_directory="DATA/chroma_db",
        embedding_function=hugg_emb_bgem3
    )

#store_bge = InMemoryStore()

full_retriever_bge = ParentDocumentRetriever(
    vectorstore=vectorstore_bgem3,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

full_retriever_bge.add_documents(data, ids = None)

In [12]:
vectorstore_bgem3.similarity_search('Is it true that Haroon Sheikh is the CEO of CareTech?')

[Document(metadata={'doc_id': '1c9f26d6-14b6-46a8-b33b-c102e347b249', 'page': 40, 'source': 'DATA/annualreport.pdf'}, page_content='number of AIM quoted and private \ncompany boards.January 2020 October 2005\nHaroon is amongst the most experienced \nCEOs in the health and social care \nsector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has'),
 Document(metadata={'doc_id': '1c9f26d6-14b6-46a8-b33b-c102e347b249', 'page': 40, 'source': 'DATA/annualreport.pdf'}, page_content='with his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated indus

In [8]:
vectorstore.similarity_search('Is it true that Haroon Sheikh is the CEO of CareTech?')

[Document(page_content='number of AIM quoted and private \ncompany boards.January 2020 October 2005\nHaroon is amongst the most experienced \nCEOs in the health and social care \nsector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has', metadata={'doc_id': '6ee008de-9630-4bbd-b07a-2ff6b0a85ea3', 'page': 40, 'source': 'DATA/annualreport.pdf'}),
 Document(page_content='with his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment', metadata={'doc_id': '6ee008de-9630-4bbd-

In [8]:
from langchain_core.retrievers import BaseRetriever
from langchain.vectorstores import VectorStore 
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List
from langchain_core.documents.base import Document
from langchain.load import dumps, loads
from langchain_core.output_parsers import JsonOutputParser

'''
Explicación del retriever, en particular la función _get_relevant_documents() para saber como usar el retriever cuando se quieran añadir más 
condiciones en el rrf: 
Esta clase está estructurada para que con las funciones rrf() y get_fused_scores() se puedan ir añadiendo tantos metodos de recuperación de 
contexto como se quiera. (Siempre que los documentos recuperados tengan un orden concreto, para los que no, ya está el método fused_scores_literal()
que asigna a todos los documentos el mismo valor). 
¿Cómo funciona y se puede automatizar la función _get_relevant_documents()?
Supongamos que tenemos n criterios de recuperación de contextos (en este ejemplo tenemos 4, literal, mxbai l2, mxbai cos, bge cos)
En primer lugar, se necestita ir calculando los scores de los documentos que vamos obteniendo para ello, se va haciendo uso de la función get_fused_scores()
scores1 =  get_fused_scores({},lista1), scores2 = get_fused_scores(scores1,lista2), ... , scoresn = get_fused_scores(scores(n-1), listan)
Esto lo que va haciendo es ir acumulando los scores de acuerdo al valor que se ha decidido dar. En este caso, 1/1+rank. 
Por ultimo una vez tengamos todos los documentos con sus scores se hace uso de la función rrf con los ultimos scores, que se encarga de dar la lista final de 
documentos ordenada sin los scores ya para que la procese la función del retriever. 
rrf(scores(n)) puesto que esta función desde dentro ya se encarga de calcular los scores para la ultima lista.
'''
class CustomRetriever_advanced(BaseRetriever):
    '''En este caso se necesitan dos db vectoriales que almacenen los embeddings que genera cada modelo diferente. Se pueden usar
    tantos modelos de generación de embeddings como se quiera simplemente habrá que pasar en esta clase del retriever personalizado 
    las bases de datos correspondientes. También se va a añadir una variable k que será el número de documentos que se quieran recuperar
    Por ejemplo si k = 10 pues el retriever generará un contexto con los 10 documentos que mayor score hayan generado. En el caso anterior 
    que k no se especificaba, en los documentos se metían todos los que se iban recuperando de cada método. '''
    vs: VectorStore
    vs_2: VectorStore
    k: int
    '''fused_scores es un diccionario que va a ir actualizando la solucion de rrf por lo que en la primera
    iteración fused_scores = {} (diccionario vacio) mientras que para las demás iteraciones será la solución
    de esta función junto con las diferentes listas que queramos ir pasandole el rrf'''
    
    '''
    Cada vez que se hace una llamada a la función rrf nos da una lista de objetos Document (sin el score) que están ya listos 
    para pasar al retrievalQA. Es necesario pasarle la lista de scores que se va obteniendo conforme se van usando técnicas en
    el reranking. 
    '''
    def rrf(self,fused_scores):
        reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
        #falta pasar estos resultados a una lista de documentos que es lo que devuelve el retriver (loads)
        lista_rerank = []
        print('Documentos finales (score)')
        for doc, score in reranked_results.items():
            lista_rerank.append(loads(doc))
            print('------')
            print(score)
        return lista_rerank
    
    '''
    La función get_fused_scores, nos sirve para ir actualizando los scores de una lista de documetnos dada. Por ejemplo, si estamos 
    en la primera iteración del rrf, previus_fused = {} y esta función devolverá un diccionario actualizado con los documetnos de la lista 
    y su correspondiente score. Este diccionario será necesario para pasarselo luego en la función rrf. 
    Cuando no estamos en la primera iteración y previus_fused ya no es un diccionario vacio, se le pasará un diccionario de documentos y 
    scores junto con una nueva lista que tendrá los documentos que queremos incorporar a la lista de scores. 
    '''   
    def get_fused_scores(self,previus_fused, lista):
        for rank, doc in enumerate(lista):
            doc_str = dumps(doc)
            if doc_str not in previus_fused:
                previus_fused[doc_str] = 0
            previus_score = previus_fused[doc_str]
            print(f'scores previos {previus_score}')
            previus_fused[doc_str] += 1 / (rank + 1)
        return previus_fused
    
    '''
    Este método realiza la misma función que la anterior solo que en esta se ha utilizado para arreglar el problema de tener doc_id diferente para el
    mismo chunk en dos bases de datos. Lo que hace es ir metiendo los documentos en una lista y actualizar el metadata de los nuevos que van entrando si 
    ya se encuentran en la lista. Asi solucionamos el tema de la duplicidad, el resto de funcionamiento es idéntico.
    '''
    def get_fused_scores_v2(self,previus_fused, lista):
        
        previus_list = []
        for doc, score in previus_fused.items():
            previus_list.append(loads(doc))
        
        for d in lista:
            for p_d in previus_list:
                if d.page_content == p_d.page_content: 
                    d.metadata['doc_id'] = p_d.metadata['doc_id']

        for rank, doc in enumerate(lista):
            doc_str = dumps(doc)
            if doc_str not in previus_fused:
                previus_fused[doc_str] = 0
            previus_score = previus_fused[doc_str]
            print(f'scores previos {previus_score}')
            previus_fused[doc_str] += 1 / (rank + 1)

        return previus_fused


    '''
    Nota: Esta función no haría falta si no se le diera la misma puntuación a cada documento obtenido en la búsqueda literal. Esto se debe a que 
    cuando se buscan documentos de forma literal todos han de tener el mismo valor. No se puede distinguir de esta manera si hay uno "mejor" que otro. 
    Como aqui tenemos fused_scores = {}, esto significa que si queremos usar esta estrategia siempre tiene que ser los primeros scores que se recopilan, 
    si no hay que cambiar y pasarle como argumetno unos scores previos y acumular con += 0.5
    En resumen no es necesario aqui comprobar lo de la lista auxiliar y cambiar el metadata de doc_id ya que en este caso serían los primeros documentos 
    que entrarían en la lista y en ningún caso estarían duplicados. 
    '''
    def fused_scores_literal(self, lista):
        fused_scores = {}
        for doc in lista:
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previus_score = fused_scores[doc_str]
            print(f'scores previos {previus_score}')
            fused_scores[doc_str] = 0.5
        return fused_scores
    ''' 
    Genera una consulta para que sea dinámica la busqueda literal en la base de datos, dada una lista de palabras clave. 
    El método de busqueda literal en la base de datos chroma, necesita el siguiente formato
    {
    "$and": [
        {"$contains": "key_word1"},
        {"$contains": "key_word2"}
        ]
    }
    Con lo cual, este método en función de las palabras clave que tenga nuestra query, se van añadiendo a la consulta y no se
    necesita ir creando una consulta a mano para cada búsqueda literal que tenga diferente numero de palabras clave. 
    '''
    def generaConsulta(self,key_words):
        busquedas = []
        for w in key_words:
            conta = {"$contains":str(w)}
            busquedas.append(conta)
        #Duda: Poner un or en vez de un and. En el caso de que existan muchos keywords puede ser dificil encontrar documentos que los contengan todas. 
        consulta = {"$and": busquedas}
        return consulta
    
    '''Corrective RAG:
    La función CRAG, recibe una query (pregunta) y una serie de documentos que han sido recuperados con el objetivo de responder la pregunta
    de forma correcta. En este caso, se crea un LLM que sea capaz de evaluar cada uno de estos documentos y decida si su contenido es adecuado
    para responder la pregunta. Los documentos que se consideren adecuados son aquellos que se devuelven en esta función. 
    '''
    def CRAG(self, query, docs):

        prompt = PromptTemplate(
            template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
            Here is the retrieved document: \n\n {document} \n\n
            Here is the user question: {question} \n
            If the document contains keywords related to the user question, grade it as relevant. \n
            It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
            Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
            Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
            input_variables=["question", "document"],
        )

        retrieval_grader = prompt | llm | JsonOutputParser()

        valid_docs = []

        for d in docs:
            score = retrieval_grader.invoke({"question": query, "document": d.page_content})
            if score['score'] == "yes":
                valid_docs.append(d)
            else:
                continue

        return valid_docs
    
    '''
    En el RetrievalQA, que se pasa en la cadena, el retriever que se usa por la clase para obtener los diferentes documentos hace uso de una función
    llamada get_relevant_documents() que viene predeterminada en el retriever base. Cuando hacemos un retriever personalizado, es necesario crear una
    función _get_relevant_documents(), con _ delante del mismo nombre anterior para que a la hora de recuperar documentos la función RetrievalQA haga 
    uso de la función que hemos personalizado para recuperar los documentos. Solo tenemos que tener en cuenta que al aplicar los cambios que queremos
    al personalizar esta función, se devuelva una lista de documentos (objetos Document)
    '''
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
        ) -> List[Document]:
        #Quitar la parte de spacy pq da error de dependencias, esto es la parte literal !!
        '''
        spacyModel = spacy.load("en_core_web_sm")
        list = self.vs.get(
            where_document=self.generaConsulta(spacyModel(query).ents)
            )
        
        literal_docs = []
      
        for i in range(len(list['ids'])):
            doc = Document(page_content=list['documents'][i],metadata=list['metadatas'][i])
            literal_docs.append(doc)
        '''
        
        #documentos
        docs_l2 = self.vs.similarity_search(query)

        docs_simcos = self.vs.similarity_search_by_vector(hugg_embeddings.embed_query(query))
        
        docs_simcos_bge = self.vs_2.similarity_search_by_vector(hugg_emb_bgem3.embed_query(query))
        #Scores acumulados de cada haciendo uso de cada tecnica
        #scores_literal = self.fused_scores_literal(literal_docs)

        scores_l2 = self.get_fused_scores_v2({},docs_l2)
        
        scores_simcos = self.get_fused_scores_v2(scores_l2,docs_simcos)

        scores_bge = self.get_fused_scores_v2(scores_simcos,docs_simcos_bge)
        #Recuperación de documentos
        rrf_documents = self.rrf(scores_bge)

        #evaluación de los documentos con CRAG.
        crag_documents = self.CRAG(query,rrf_documents)

        ids = []

        for d in crag_documents: 
            if d.metadata['doc_id'] not in ids: 
                ids.append(d.metadata['doc_id'])

        parent_docs = store.mget(ids)

        str_docs = []
        docs = []
        for pd in parent_docs:
            parent_doc_str = dumps(pd)
            if parent_doc_str not in str_docs:
                if pd is not None:
                    str_docs.append(parent_doc_str)
                    docs.append(loads(parent_doc_str))

        return docs[0:self.k]

In [9]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    It is mandatorian that only if the answer is not in the context, answer "I have not enough context in order to answer this" and stop the answer.
    Try to use the memory context in the answer only if the question mentions it.
"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=CustomRetriever_advanced(vs = vectorstore, vs_2=vectorstore_bgem3, k=4),
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)
query = 'Is it true that Haroon Sheikh is the CEO of CareTech?'
result = qa_chain({"query": query})
result



> Entering new RetrievalQA chain...
scores previos 0
scores previos 0
scores previos 0
scores previos 0
scores previos 0.3333333333333333
scores previos 1.3333333333333333
scores previos 1.8333333333333333
scores previos 0.25
scores previos 2.1666666666666665
scores previos 3.1666666666666665
scores previos 0.5
scores previos 0.8333333333333333
Documentos finales (score)
------
3.6666666666666665
------
1.0833333333333333
------
1.0
------
0.5
 {"score": "yes"} {
"score": "yes"
} {"score": "yes"} {"score": "yes"}

> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: and Robert W. Baird for two years until 
June 2002. Karl set up Ashling Capital 
LLP in December 2002 to provide 
consultancy services to quoted 
and private companies. He sits on a 
number of AIM quoted and private 
company bo

{'query': 'Is it true that Haroon Sheikh is the CEO of CareTech?',
 'result': ' Yes, Haroon Sheikh is the Group CEO of CareTech. He leads the day-to-day running of the Group and has been instrumental in its international expansion, as stated in the context you provided.'}

In [10]:
qa_chain({"query": query})['result']



> Entering new RetrievalQA chain...
scores previos 0
scores previos 0
scores previos 0
scores previos 0
scores previos 0.3333333333333333
scores previos 1.3333333333333333
scores previos 1.8333333333333333
scores previos 0.25
scores previos 2.1666666666666665
scores previos 3.1666666666666665
scores previos 0.5
scores previos 0.8333333333333333
Documentos finales (score)
------
3.6666666666666665
------
1.0833333333333333
------
1.0
------
0.5
 {"score": "yes"} {
"score": "yes"
} {"score": "yes"} {"score": "yes"}

> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: and Robert W. Baird for two years until 
June 2002. Karl set up Ashling Capital 
LLP in December 2002 to provide 
consultancy services to quoted 
and private companies. He sits on a 
number of AIM quoted and private 
company bo

' Yes, Haroon Sheikh is the Group CEO of CareTech. He leads the day-to-day running of the Group and has been instrumental in its international expansion, as stated in the context you provided.'